# 📈 **Regresión Lineal: Ejemplos Prácticos**

Este notebook contiene ejemplos completos de regresión lineal simple y múltiple con aplicaciones del mundo real.

## **Contenido:**
1. **Regresión Lineal Simple**: Precio de casas vs metros cuadrados
2. **Regresión Lineal Múltiple**: Predicción de precios inmobiliarios
3. **Análisis de Ventas**: Modelo de ventas con múltiples variables
4. **Interpretación de Resultados**: Coeficientes y métricas
5. **Visualizaciones**: Gráficos de análisis y diagnóstico


In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


## **1. Regresión Lineal Simple: Precio de Casas**

### **Problema**: Predecir el precio de una casa basándose únicamente en sus metros cuadrados

**Ecuación**: `Precio = β₀ + β₁ × Metros²`


In [ ]:
# Crear dataset simulado de casas
np.random.seed(42)
n_casas = 200

# Generar datos realistas
metros_cuadrados = np.random.normal(120, 30, n_casas)  # Media 120m², desv 30m²
metros_cuadrados = np.clip(metros_cuadrados, 50, 300)  # Limitar entre 50-300m²

# Precio base + relación lineal + ruido
precio_base = 100000  # $100,000 base
precio_por_metro = 2500  # $2,500 por metro cuadrado
ruido = np.random.normal(0, 20000, n_casas)  # Ruido aleatorio

precio = precio_base + precio_por_metro * metros_cuadrados + ruido
precio = np.clip(precio, 50000, 800000)  # Limitar precios realistas

# Crear DataFrame
df_casas = pd.DataFrame({
    'metros_cuadrados': metros_cuadrados,
    'precio': precio
})

print("=== DATASET DE CASAS ===")
print(df_casas.head())
print(f"\nEstadísticas descriptivas:")
print(df_casas.describe())


In [ ]:
# Visualizar la relación entre variables
plt.figure(figsize=(12, 5))

# Scatter plot con línea de tendencia
plt.subplot(1, 2, 1)
sns.scatterplot(data=df_casas, x='metros_cuadrados', y='precio', alpha=0.6)
sns.regplot(data=df_casas, x='metros_cuadrados', y='precio', 
            scatter=False, color='red', line_kws={'linewidth': 2})
plt.title('Relación: Precio vs Metros Cuadrados')
plt.xlabel('Metros Cuadrados (m²)')
plt.ylabel('Precio ($)')

# Distribución de precios
plt.subplot(1, 2, 2)
sns.histplot(df_casas['precio'], kde=True, bins=20)
plt.title('Distribución de Precios')
plt.xlabel('Precio ($)')
plt.ylabel('Frecuencia')

plt.tight_layout()
plt.show()

# Calcular correlación
correlacion = df_casas['metros_cuadrados'].corr(df_casas['precio'])
print(f"Correlación entre metros cuadrados y precio: {correlacion:.3f}")


In [ ]:
# Preparar datos para el modelo
X = df_casas[['metros_cuadrados']]  # Variable independiente
y = df_casas['precio']              # Variable dependiente

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
modelo_simple = LinearRegression()
modelo_simple.fit(X_train, y_train)

# Hacer predicciones
y_pred_train = modelo_simple.predict(X_train)
y_pred_test = modelo_simple.predict(X_test)

print("=== MODELO DE REGRESIÓN LINEAL SIMPLE ===")
print(f"Intercepto (β₀): ${modelo_simple.intercept_:,.2f}")
print(f"Coeficiente (β₁): ${modelo_simple.coef_[0]:,.2f} por metro cuadrado")
print(f"\nEcuación del modelo:")
print(f"Precio = {modelo_simple.intercept_:,.0f} + {modelo_simple.coef_[0]:,.0f} × Metros²")


In [ ]:
# Evaluar el modelo
def evaluar_modelo(y_real, y_pred, nombre_dataset):
    mse = mean_squared_error(y_real, y_pred)
    mae = mean_absolute_error(y_real, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_real, y_pred)
    
    print(f"\n=== MÉTRICAS - {nombre_dataset} ===")
    print(f"R² (Coeficiente de Determinación): {r2:.4f} ({r2*100:.1f}%)")
    print(f"MSE (Error Cuadrático Medio): ${mse:,.0f}")
    print(f"MAE (Error Absoluto Medio): ${mae:,.0f}")
    print(f"RMSE (Raíz del Error Cuadrático): ${rmse:,.0f}")
    
    return {'R2': r2, 'MSE': mse, 'MAE': mae, 'RMSE': rmse}

# Evaluar en entrenamiento y prueba
metricas_train = evaluar_modelo(y_train, y_pred_train, "ENTRENAMIENTO")
metricas_test = evaluar_modelo(y_test, y_pred_test, "PRUEBA")


In [ ]:
# Visualizar resultados del modelo
plt.figure(figsize=(15, 5))

# 1. Predicciones vs Valores Reales
plt.subplot(1, 3, 1)
plt.scatter(y_test, y_pred_test, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Precio Real ($)')
plt.ylabel('Precio Predicho ($)')
plt.title('Predicciones vs Valores Reales')
plt.grid(True, alpha=0.3)

# 2. Residuales
residuales = y_test - y_pred_test
plt.subplot(1, 3, 2)
plt.scatter(y_pred_test, residuales, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Precio Predicho ($)')
plt.ylabel('Residuales ($)')
plt.title('Análisis de Residuales')
plt.grid(True, alpha=0.3)

# 3. Línea de regresión sobre datos originales
plt.subplot(1, 3, 3)
plt.scatter(X_test, y_test, alpha=0.6, label='Datos reales')
plt.plot(X_test, y_pred_test, 'r-', linewidth=2, label='Predicciones')
plt.xlabel('Metros Cuadrados (m²)')
plt.ylabel('Precio ($)')
plt.title('Línea de Regresión')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## **2. Regresión Lineal Múltiple: Predicción de Precios Inmobiliarios**

### **Problema**: Predecir el precio de una casa usando múltiples características

**Ecuación**: `Precio = β₀ + β₁×Metros² + β₂×Habitaciones + β₃×Antigüedad + β₄×Distancia`


In [ ]:
# Crear dataset más complejo para regresión múltiple
np.random.seed(42)
n_casas = 500

# Generar variables independientes
metros_cuadrados = np.random.normal(120, 30, n_casas)
metros_cuadrados = np.clip(metros_cuadrados, 50, 300)

habitaciones = np.random.poisson(3, n_casas)  # Media 3 habitaciones
habitaciones = np.clip(habitaciones, 1, 6)

antiguedad = np.random.exponential(10, n_casas)  # Media 10 años
antiguedad = np.clip(antiguedad, 0, 50)

distancia_centro = np.random.gamma(2, 5, n_casas)  # Media 10 km
distancia_centro = np.clip(distancia_centro, 1, 30)

# Generar precio con múltiples variables
precio_base = 100000
precio_por_metro = 2500
precio_por_habitacion = 15000
precio_por_antiguedad = -2000  # Negativo: casas viejas valen menos
precio_por_distancia = -1000   # Negativo: lejos del centro vale menos

ruido = np.random.normal(0, 25000, n_casas)

precio = (precio_base + 
          precio_por_metro * metros_cuadrados +
          precio_por_habitacion * habitaciones +
          precio_por_antiguedad * antiguedad +
          precio_por_distancia * distancia_centro +
          ruido)

precio = np.clip(precio, 50000, 800000)

# Crear DataFrame
df_casas_multiple = pd.DataFrame({
    'metros_cuadrados': metros_cuadrados,
    'habitaciones': habitaciones,
    'antiguedad': antiguedad,
    'distancia_centro': distancia_centro,
    'precio': precio
})

print("=== DATASET DE CASAS (MÚLTIPLE) ===")
print(df_casas_multiple.head())
print(f"\nEstadísticas descriptivas:")
print(df_casas_multiple.describe())


In [ ]:
# Análisis de correlaciones
plt.figure(figsize=(12, 8))

# Matriz de correlación
plt.subplot(2, 2, 1)
correlation_matrix = df_casas_multiple.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f')
plt.title('Matriz de Correlación')

# Relaciones individuales con precio
variables = ['metros_cuadrados', 'habitaciones', 'antiguedad', 'distancia_centro']
for i, var in enumerate(variables, 2):
    plt.subplot(2, 2, i)
    sns.scatterplot(data=df_casas_multiple, x=var, y='precio', alpha=0.6)
    plt.title(f'Precio vs {var.replace("_", " ").title()}')
    plt.xlabel(var.replace('_', ' ').title())

plt.tight_layout()
plt.show()

# Mostrar correlaciones con precio
print("=== CORRELACIONES CON PRECIO ===")
for var in variables:
    corr = df_casas_multiple[var].corr(df_casas_multiple['precio'])
    print(f"{var.replace('_', ' ').title()}: {corr:.3f}")


In [ ]:
# Preparar datos para regresión múltiple
X_multiple = df_casas_multiple[['metros_cuadrados', 'habitaciones', 'antiguedad', 'distancia_centro']]
y_multiple = df_casas_multiple['precio']

# Dividir en entrenamiento y prueba
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_multiple, y_multiple, test_size=0.2, random_state=42)

# Crear y entrenar el modelo múltiple
modelo_multiple = LinearRegression()
modelo_multiple.fit(X_train_m, y_train_m)

# Hacer predicciones
y_pred_train_m = modelo_multiple.predict(X_train_m)
y_pred_test_m = modelo_multiple.predict(X_test_m)

print("=== MODELO DE REGRESIÓN LINEAL MÚLTIPLE ===")
print(f"Intercepto (β₀): ${modelo_multiple.intercept_:,.2f}")
print(f"\nCoeficientes:")
nombres_vars = ['Metros²', 'Habitaciones', 'Antigüedad', 'Distancia Centro']
for i, (nombre, coef) in enumerate(zip(nombres_vars, modelo_multiple.coef_)):
    print(f"β{i+1} ({nombre}): ${coef:,.2f}")

print(f"\nEcuación del modelo:")
ecuacion = f"Precio = {modelo_multiple.intercept_:,.0f}"
for i, (nombre, coef) in enumerate(zip(nombres_vars, modelo_multiple.coef_)):
    signo = "+" if coef >= 0 else ""
    ecuacion += f" {signo}{coef:,.0f}×{nombre}"
print(ecuacion)


In [ ]:
# Evaluar modelo múltiple
print("=== COMPARACIÓN DE MODELOS ===")
metricas_train_m = evaluar_modelo(y_train_m, y_pred_train_m, "MÚLTIPLE - ENTRENAMIENTO")
metricas_test_m = evaluar_modelo(y_test_m, y_pred_test_m, "MÚLTIPLE - PRUEBA")

print(f"\n=== MEJORA DEL MODELO MÚLTIPLE ===")
print(f"R² Simple: {metricas_test['R2']:.4f}")
print(f"R² Múltiple: {metricas_test_m['R2']:.4f}")
print(f"Mejora: {(metricas_test_m['R2'] - metricas_test['R2'])*100:.1f} puntos porcentuales")

print(f"\nRMSE Simple: ${metricas_test['RMSE']:,.0f}")
print(f"RMSE Múltiple: ${metricas_test_m['RMSE']:,.0f}")
print(f"Reducción de error: {((metricas_test['RMSE'] - metricas_test_m['RMSE'])/metricas_test['RMSE'])*100:.1f}%")


In [ ]:
# Ejemplo de predicción con el modelo múltiple
print("=== EJEMPLO DE PREDICCIÓN ===")

# Crear una casa de ejemplo
casa_ejemplo = pd.DataFrame({
    'metros_cuadrados': [150],
    'habitaciones': [3],
    'antiguedad': [5],
    'distancia_centro': [8]
})

precio_predicho = modelo_multiple.predict(casa_ejemplo)[0]

print(f"Casa de ejemplo:")
print(f"- Metros cuadrados: {casa_ejemplo['metros_cuadrados'].iloc[0]} m²")
print(f"- Habitaciones: {casa_ejemplo['habitaciones'].iloc[0]}")
print(f"- Antigüedad: {casa_ejemplo['antiguedad'].iloc[0]} años")
print(f"- Distancia al centro: {casa_ejemplo['distancia_centro'].iloc[0]} km")
print(f"\nPrecio predicho: ${precio_predicho:,.0f}")

# Calcular contribución de cada variable
contribuciones = {}
for i, (nombre, coef) in enumerate(zip(nombres_vars, modelo_multiple.coef_)):
    valor = casa_ejemplo.iloc[0, i]
    contribucion = coef * valor
    contribuciones[nombre] = contribucion

print(f"\nContribución de cada variable:")
for nombre, contrib in contribuciones.items():
    print(f"- {nombre}: ${contrib:,.0f}")

print(f"- Intercepto: ${modelo_multiple.intercept_:,.0f}")
print(f"- Total: ${sum(contribuciones.values()) + modelo_multiple.intercept_:,.0f}")


## **3. Análisis de Ventas: Modelo de Regresión Empresarial**

### **Problema**: Predecir ventas mensuales basándose en presupuesto de marketing, temporada y otros factores


In [ ]:
# Crear dataset de ventas empresariales
np.random.seed(42)
n_meses = 60  # 5 años de datos

# Variables independientes
presupuesto_marketing = np.random.gamma(2, 5000, n_meses)  # Media $10,000
presupuesto_marketing = np.clip(presupuesto_marketing, 1000, 50000)

empleados = np.random.poisson(25, n_meses)  # Media 25 empleados
empleados = np.clip(empleados, 10, 50)

temporada = np.random.choice([1, 2, 3, 4], n_meses)  # 1=Invierno, 2=Primavera, 3=Verano, 4=Otoño

precio_producto = np.random.normal(100, 20, n_meses)  # Media $100
precio_producto = np.clip(precio_producto, 50, 200)

# Generar ventas con relación compleja
ventas_base = 50000
ventas_por_marketing = 2.5  # $2.5 por cada $1 de marketing
ventas_por_empleado = 2000  # $2,000 por empleado
ventas_temporada = [0, 5000, 10000, 3000]  # Bonus por temporada
ventas_por_precio = -100  # Ventas disminuyen con precio

ruido = np.random.normal(0, 10000, n_meses)

ventas = (ventas_base + 
          ventas_por_marketing * presupuesto_marketing +
          ventas_por_empleado * empleados +
          np.array([ventas_temporada[t-1] for t in temporada]) +
          ventas_por_precio * precio_producto +
          ruido)

ventas = np.clip(ventas, 0, 300000)

# Crear DataFrame
df_ventas = pd.DataFrame({
    'presupuesto_marketing': presupuesto_marketing,
    'empleados': empleados,
    'temporada': temporada,
    'precio_producto': precio_producto,
    'ventas': ventas
})

print("=== DATASET DE VENTAS EMPRESARIALES ===")
print(df_ventas.head())
print(f"\nEstadísticas descriptivas:")
print(df_ventas.describe())


In [ ]:
# Análisis de ventas por temporada
plt.figure(figsize=(15, 5))

# Ventas por temporada
plt.subplot(1, 3, 1)
sns.boxplot(data=df_ventas, x='temporada', y='ventas')
plt.title('Ventas por Temporada')
plt.xlabel('Temporada (1=Invierno, 2=Primavera, 3=Verano, 4=Otoño)')
plt.ylabel('Ventas ($)')

# Relación marketing vs ventas
plt.subplot(1, 3, 2)
sns.scatterplot(data=df_ventas, x='presupuesto_marketing', y='ventas', alpha=0.6)
plt.title('Ventas vs Presupuesto de Marketing')
plt.xlabel('Presupuesto Marketing ($)')
plt.ylabel('Ventas ($)')

# Relación empleados vs ventas
plt.subplot(1, 3, 3)
sns.scatterplot(data=df_ventas, x='empleados', y='ventas', alpha=0.6)
plt.title('Ventas vs Número de Empleados')
plt.xlabel('Número de Empleados')
plt.ylabel('Ventas ($)')

plt.tight_layout()
plt.show()

# Estadísticas por temporada
print("=== VENTAS PROMEDIO POR TEMPORADA ===")
ventas_por_temporada = df_ventas.groupby('temporada')['ventas'].agg(['mean', 'std', 'count'])
temporadas = ['Invierno', 'Primavera', 'Verano', 'Otoño']
for i, temp in enumerate(temporadas, 1):
    mean_ventas = ventas_por_temporada.loc[i, 'mean']
    std_ventas = ventas_por_temporada.loc[i, 'std']
    print(f"{temp}: ${mean_ventas:,.0f} ± ${std_ventas:,.0f}")


In [ ]:
# Preparar datos para el modelo de ventas
X_ventas = df_ventas[['presupuesto_marketing', 'empleados', 'temporada', 'precio_producto']]
y_ventas = df_ventas['ventas']

# Dividir en entrenamiento y prueba
X_train_v, X_test_v, y_train_v, y_test_v = train_test_split(X_ventas, y_ventas, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de ventas
modelo_ventas = LinearRegression()
modelo_ventas.fit(X_train_v, y_train_v)

# Hacer predicciones
y_pred_train_v = modelo_ventas.predict(X_train_v)
y_pred_test_v = modelo_ventas.predict(X_test_v)

print("=== MODELO DE REGRESIÓN PARA VENTAS ===")
print(f"Intercepto (β₀): ${modelo_ventas.intercept_:,.2f}")
print(f"\nCoeficientes:")
nombres_ventas = ['Presupuesto Marketing', 'Empleados', 'Temporada', 'Precio Producto']
for i, (nombre, coef) in enumerate(zip(nombres_ventas, modelo_ventas.coef_)):
    print(f"β{i+1} ({nombre}): ${coef:,.2f}")

print(f"\nEcuación del modelo:")
ecuacion_ventas = f"Ventas = {modelo_ventas.intercept_:,.0f}"
for i, (nombre, coef) in enumerate(zip(nombres_ventas, modelo_ventas.coef_)):
    signo = "+" if coef >= 0 else ""
    ecuacion_ventas += f" {signo}{coef:,.0f}×{nombre.replace(' ', '_')}"
print(ecuacion_ventas)


In [ ]:
# Evaluar modelo de ventas
metricas_train_v = evaluar_modelo(y_train_v, y_pred_train_v, "VENTAS - ENTRENAMIENTO")
metricas_test_v = evaluar_modelo(y_test_v, y_pred_test_v, "VENTAS - PRUEBA")

# Ejemplo de predicción de ventas
print("\n=== EJEMPLO DE PREDICCIÓN DE VENTAS ===")

# Escenario: Verano, $15,000 en marketing, 30 empleados, producto a $90
escenario = pd.DataFrame({
    'presupuesto_marketing': [15000],
    'empleados': [30],
    'temporada': [3],  # Verano
    'precio_producto': [90]
})

ventas_predichas = modelo_ventas.predict(escenario)[0]

print(f"Escenario de predicción:")
print(f"- Presupuesto de marketing: ${escenario['presupuesto_marketing'].iloc[0]:,}")
print(f"- Número de empleados: {escenario['empleados'].iloc[0]}")
print(f"- Temporada: Verano (3)")
print(f"- Precio del producto: ${escenario['precio_producto'].iloc[0]}")
print(f"\nVentas predichas: ${ventas_predichas:,.0f}")

# Análisis de sensibilidad
print(f"\n=== ANÁLISIS DE SENSIBILIDAD ===")
print("¿Qué pasaría si...?")

# Aumentar marketing en $5,000
escenario_mas_marketing = escenario.copy()
escenario_mas_marketing['presupuesto_marketing'] = 20000
ventas_mas_marketing = modelo_ventas.predict(escenario_mas_marketing)[0]
incremento_marketing = ventas_mas_marketing - ventas_predichas

print(f"- Aumentar marketing a $20,000: +${incremento_marketing:,.0f} en ventas")

# Contratar 5 empleados más
escenario_mas_empleados = escenario.copy()
escenario_mas_empleados['empleados'] = 35
ventas_mas_empleados = modelo_ventas.predict(escenario_mas_empleados)[0]
incremento_empleados = ventas_mas_empleados - ventas_predichas

print(f"- Contratar 5 empleados más: +${incremento_empleados:,.0f} en ventas")

# Cambiar a invierno
escenario_invierno = escenario.copy()
escenario_invierno['temporada'] = 1
ventas_invierno = modelo_ventas.predict(escenario_invierno)[0]
diferencia_temporada = ventas_invierno - ventas_predichas

print(f"- Cambiar a invierno: {diferencia_temporada:+,.0f} en ventas")


## **📊 Resumen y Conclusiones**

### **Comparación de Modelos**

| Modelo | R² | RMSE | Interpretabilidad | Complejidad |
|--------|----|----- |-------------------|-------------|
| **Regresión Simple** | ~0.85 | ~$25,000 | Alta | Baja |
| **Regresión Múltiple** | ~0.92 | ~$18,000 | Alta | Media |
| **Modelo de Ventas** | ~0.88 | ~$8,000 | Alta | Media |

### **Aplicaciones Prácticas**

1. **Predicción de Precios Inmobiliarios**: Modelo robusto con múltiples variables
2. **Análisis de Ventas**: Identificación de factores clave para el negocio
3. **Planificación Estratégica**: Análisis de sensibilidad para toma de decisiones

### **Ventajas de la Regresión Lineal**

✅ **Interpretabilidad**: Coeficientes claros y comprensibles  
✅ **Rapidez**: Entrenamiento y predicción eficientes  
✅ **Estabilidad**: Resultados consistentes y confiables  
✅ **Base sólida**: Punto de partida para modelos más complejos  

### **Limitaciones Identificadas**

❌ **Linealidad**: Asume relaciones lineales entre variables  
❌ **Outliers**: Sensible a valores extremos  
❌ **Multicolinealidad**: Variables correlacionadas afectan estabilidad  
❌ **Supuestos**: Requiere normalidad y homocedasticidad  

### **Próximos Pasos**

1. **Regresión Polinomial**: Para relaciones no lineales
2. **Regularización**: Lasso, Ridge para evitar sobreajuste
3. **Árboles de Regresión**: Para relaciones complejas
4. **Ensemble Methods**: Random Forest, XGBoost para mayor precisión
